In [1]:
import numpy as np
import pandas as pd
import itertools
import time
import matplotlib.pyplot as plt


In [2]:
#Activation Functions
def tan_hyp(y) :
    return np.tanh(y)

def softmax(y) :
    return np.exp(y-max(y))/np.sum(np.exp(y-max(y)))

def linear(y) : 
    return y

#Gradients

def der_softmax(y) : 
    a = np.zeros((len(y),len(y)))
    np.fill_diagonal(a,y)
    y = np.reshape(y,(len(y),-1))
    b = np.dot(y,np.transpose(y))
    return a - b

def der_tan_hyp(y) : 
    return (1-y)*(1+y)

def der_linear(y) :
    a = np.zeros((len(y),len(y)))
    np.fill_diagonal(a,y)
    return a

In [3]:
#Loss functions
def mse(y,t) : 
    return 0.5*np.sum((y-t)**2)

def cross_entropy(y,t):
    return -np.sum(t*np.log(y))

def grad_cross_entropy(y,t) :
    return -np.divide(t,y)

def grad_mse(y,t) : 
    return y-t

In [4]:
def fwpass_customweights(activation_functions,training_sample,w) : 
        a_list = [training_sample]
        h_list = [training_sample]
        output = training_sample
        
        for (activation_function,weight) in zip(activation_functions,w) :
            
            output = np.append(output,np.ones((1,1)),axis = 0)
            output = np.dot(weight,output)
            a_list.append(output)
            
            output = eval(activation_function)(output)
            h_list.append(output)
        return a_list , h_list

In [5]:
class neural_network :
    
    def __init__(self, n_hidden,list_nodes,loss_fn,list_activation_functions,learning_rate,momentum) :
        self.n_hidden = n_hidden
        self.list_nodes = list_nodes
        self.loss_fn = loss_fn
        self.list_activation_functions = list_activation_functions
        self.learning_rate = learning_rate
        self.momentum = momentum
        self.weights = []
        self.grad_past = []
        
        for i in range(n_hidden+1) :
            np.random.seed(42)
            weight_layer = np.random.randn(list_nodes[i+1],list_nodes[i]+1)
            grad_past_layer = np.zeros((list_nodes[i+1],list_nodes[i]+1))
            self.weights.append(weight_layer)
            self.grad_past.append(grad_past_layer)


In [6]:
def forward_pass(self,training_sample):
        a_list = [training_sample]
        h_list = [training_sample]
        output = training_sample
        
        for (activation_function,weight) in zip(self.list_activation_functions,self.weights) :
            
            output = np.append(output,np.ones((1,1)),axis = 0)
            output = np.dot(weight,output)
            a_list.append(output)
            
            output = eval(activation_function)(output)
            h_list.append(output)
        return a_list , h_list

In [7]:
    
    def calc_loss(self,calc_output,actual_output) :
        return eval(self.loss_fn)(calc_output,actual_output)
    
    def backward_pass(self,h_list,intermediate_loss,actual_output) : 
        '''implementation of backpropagation'''
        
        der_output_act = eval('der_'+self.list_activation_functions[-1])(h_list[-1])
        grad_lf = eval('grad_'+self.loss_fn)(h_list[-1],actual_output)
        grad_layer = np.dot(der_output_act,grad_lf)
                
        for i in range(len(self.weights)) : 
            
            grad_weight_layer = np.dot(grad_layer,np.transpose(np.append(h_list[len(h_list)-i-2],np.ones((1,1)),axis = 0)))
            
            grad_prev_output = np.dot(np.transpose(self.weights[len(self.weights)-i-1])[:-1,:],grad_layer)
            grad_layer = grad_prev_output * eval('der_'+self.list_activation_functions[len(self.list_activation_functions)-i-2])(h_list[len(h_list)-i-2])
            
            self.weights[len(self.weights)-i-1] = self.gen_delta_rule(self.weights[len(self.weights)-i-1],grad_weight_layer,self.grad_past[len(self.grad_past)-i-1]) 
            self.grad_past[len(self.grad_past)-i-1] = self.learning_rate*grad_weight_layer
            
    def gen_delta_rule(self,weights_present,grad_present,weight_change_previous) :
        return weights_present - (grad_present*self.learning_rate) - (weight_change_previous*self.momentum)


In [12]:
def train(self,input_array,output_array,valid_input_array,valid_output_array) : 
        loss_list = []
        valid_loss_list = []
        epoch_num = 0
        prev_loss_avg = 0
        loss_avg = 0
        
        while (epoch_num<=0 or abs(prev_loss_avg - loss_avg)>=1e-5) : 
            
            prev_loss_avg = loss_avg
            loss_avg = 0
                
            print('Epoch Number : ',epoch_num)
            print('Started')
            print('----------------------------------------')
            for (training_sample,actual_output) in zip(input_array,output_array) :
                
                a_list,h_list = self.forward_pass(training_sample)
                
                intermediate_loss = self.calc_loss(h_list[-1],actual_output)
                
                self.backward_pass(h_list,intermediate_loss,actual_output)
                
                loss_avg = loss_avg + intermediate_loss
            
            loss_avg = loss_avg/len(input_array)
            loss_list.append(loss_avg)
            
            valid_loss_avg = 0
            
            for (training_sample,actual_output) in zip(valid_input_array,valid_output_array) :
                
                a_list,h_list = self.forward_pass(training_sample)
                
                intermediate_loss = self.calc_loss(h_list[-1],actual_output)
                                
                valid_loss_avg = valid_loss_avg + intermediate_loss
            
            valid_loss_avg = valid_loss_avg/len(valid_input_array)
            valid_loss_list.append(valid_loss_avg)
            
            print('Average Loss : ',loss_avg)    
            print('----------------------------------------')
            epoch_num = epoch_num + 1
            
            return loss_list,valid_loss_list
            
def test(self,input_array,output_array) : 
        accuracy = 0
        for (sample,output) in zip(input_array,output_array) : 
            
            a_list,h_list = self.forward_pass(sample)
            calc_output = h_list[-1]
            
            if np.where(calc_output == np.amax(calc_output)) == np.where(output == np.amax(output)) :
                accuracy = accuracy + 1
            
        accuracy = accuracy / len(input_array)    
        return accuracy 

In [ ]:
data = None
input_array = []
output_array = []

test_input_array = []
train_input_array= []

data = pd.read_csv('<enter csv file>')

for i in range(len(data)) :
    input_array.append(np.array([data['x1'][i],data['x2'][i]]).reshape(2,1))
    if data['label'][i] == 0 :
        output_array.append(np.array([1.0,0.0,0.0]).reshape(3,1))
    if data['label'][i] == 1 :
        output_array.append(np.array([0.0,1.0,0.0]).reshape(3,1))
    if data['label'][i] == 2 :
        output_array.append(np.array([0.0,0.0,1.0]).reshape(3,1))  

test_input_array = input_array[int(np.floor(len(data)*3/4)):]
test_output_array = output_array[int(np.floor(len(data)*3/4)):]

train_input_array = input_array[:int(np.floor(len(data)*3/4))]
train_output_array = output_array[:int(np.floor(len(data)*3/4))]

In [ ]:
nn = neural_network(2,[2,5,5,3],'cross_entropy',['tan_hyp','tan_hyp','softmax'],0.001,0.8)

list_avg_error,list_valid_avg_error = nn.train(train_input_array,train_output_array,test_input_array,test_output_array)

accuracy = nn.test(test_input_array,test_output_array)

print("Training Finished")
print("---------------------------------------------")
print("Accuracy : ",accuracy)